# Projeto 1 - Ciência dos Dados

Nome: Alexia Barbosa Pires

Nome: Ellen Coutinho Lião da Silva

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [245]:
%matplotlib inline
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import re
import os

In [246]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Alexia\Documents\2° semestre 22.2\CDados\Projetos\p1_alexia_ellen\22-2a-cd-p1-grupo_alexiabp\notebooks


Carregando a base de dados com as notícias classificadas manualmente:

In [247]:
path = pathlib.Path(r"..\data")
filename = f'{path}\dados.xlsx'

In [248]:
#Base de TREINAMENTO:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,financas,CORREÇÃO-Dólar caminha para 4ª alta semanal se...,(Corrige no 9º parágrafo para “outubro” no lug...,13/05/2022 12:04,71,1,NaN,NaN,NaN
1,financas,"Ouro fecha em alta, após quedas recentes, com ...",O contrato mais líquido do ouro fechou em alta...,03/05/2022 15:16,78,2,NaN,NaN,NaN
2,financas,"Bolsas da Europa fecham em queda, sem resoluçã...",As principais bolsas europeias fecharam em que...,10/03/2022 14:26,110,0,NaN,NaN,NaN
3,economia,"Aneel aprova reajuste médio de 15,77% nas tari...",A Agência Nacional de Energia Elétrica (Aneel)...,23/08/2022 11:21,57,0,NaN,NaN,NaN
4,financas,Dólar reduz queda com fortalecimento do DXY ap...,O dólar reduziu a queda intradia frente ao rea...,18/08/2022 10:33,12,1,NaN,NaN,NaN


In [249]:
#BASE DE TESTES:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head()

C:\Users\Alexia\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Alexia\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Categoria,Titulo,Descrição,Data,Pagina,Target,Unnamed: 6
0,financas,Ouro fecha em alta com câmbio favorável a comm...,"O ouro fechou em alta nesta terça-feira, 9, ap...",09/11/2021 15:46,185,2,NaN
1,financas,"Bolsas da Europa fecham em queda, com comentár...",Os principais mercados acionários da Europa fe...,29/08/2022 13:33,6,2,NaN
2,financas,Juros ficam estáveis com mercado ponderando câ...,Os juros terminaram a sessão regular praticame...,23/03/2022 18:00,101,2,NaN
3,financas,Juros: Alívio externo e queda do dólar abrem e...,"Os juros futuros terminaram a segunda-feira, 2...",25/07/2022 17:38,26,2,NaN
4,financas,"Ibovespa sobe 0,31%, a 115,1 mil pontos, melho...",Mesmo com a retomada da percepção de risco no ...,16/02/2022 18:44,124,2,NaN


___
## Classificador automático


O nosso tema é a relevância da alta e da queda da moeda americana, o Dólar! Pegamos nossa base de dados no site MoneyTimes, extraímo se classificamos manualmente cerca de 1048 notícias acerca deste tema.

Utilizamos a seguinte classificação:
- Target 0 = neutro
- Target 1 = positivo
- Target 2 = negativo

Notícias com a target 0 são noticias que não mencionam a palavra dólar, então classificamos como neutra/irrelevantes
Notícias com target 1 são noticias que mencionam sobre a alta/valorização da moeda no cenário mundial
Notícias com target 2 são noticias que falam sobre a queda/desvalorização da moeda no cenário mundial

O Classificador Naive-Bayes, neste projeto, consiste em categorizar de maneira automática uma base de notícias de acordo com o tema "Dólar". Nós oferecemos uma base com diversas notícias e falamos: classificador, de acordo com esse monte de palavras que eu te dei e com cada categoria que eu te informei, me traga a probalidade de cada palavra aparecer no todo. Depois disso, classifica automaticamente outras palavras de acordo com a categoria, baseando-se nessa probabilidade.

___
## Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, vamos ensinar nosso classificador.

#### 1°) Vamos fazer uma limpeza no título e na descrição de cada notícia 
Lembrando, nossas targets são:
- Target 0 = neutro
- Target 1 = positivo
- Target 2 = negativo

In [299]:
#Limpando o texto
def cleanup(text):
    #import string
    text = text.replace('“','')
    text = text.replace('[…]','')
    punctuation = '[!-.:?;\/–]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

#### 2º) Para melhorar a análise do classificador, utilizaremos o método de stopwords

Essa metodologia se faz necessária já que há excesso de palavras que não são relevantes para os resultados.

In [300]:
stopwords = [
    
    'de','da','a','à','o','em','do', 'as', 'às', 'dos', 'das', 'no', 'na', 'quem', 'ao', 'que', 'e', 'sua','seu', 'por', 'uma', 
    'um', 'os', 'nesta','neste','nesse','nessa', 'aos', 'com', 'já', 'para', 'dólar'
            
            ]

#Dólar como limpeza avançada

In [301]:
#Juntando Título e descrição do target 0
target_0 = test.loc[train['Target']==0,:]
string_0 = ''
for texto in target_0['Descrição']:
    novo_texto=cleanup(str(texto).lower())
    string_0+=''.join(novo_texto)
    
for titulo in target_0['Titulo']:
    novo_titulo=cleanup(str(titulo).lower())
    string_0+=''.join(novo_titulo)
    
lista_0 = string_0.split()

#Metodologia Stopwords

i = 0
while i<len(lista_0):
    if lista_0[i] == 'alta' or lista_0[i] == 'queda':
        del lista_0[i]
    #Se a palavra estiver na lista de stopwords, deleta
    if lista_0[i] in stopwords:
        del lista_0[i]
        
        if i != 0:
            i-=1
    else:
        i+=1
lista_0

['juros',
 'terminaram',
 'sessão',
 'regular',
 'praticamente',
 'estáveis',
 'viés',
 'baixa',
 'nos',
 'longos',
 'nos',
 'curtos',
 'forte',
 'ante',
 'real',
 'recuo',
 'rendimento',
 'treasuries',
 'deram',
 'suporte',
 'movimento',
 'curva',
 'partir',
 'miolo',
 'durante',
 'boa',
 'parte',
 'sessão',
 'apesar',
 'nova',
 'escalada',
 'preços',
 'juros',
 'futuros',
 'terminaram',
 'segundafeira',
 '25',
 'continuidade',
 'movimento',
 'alívio',
 'prêmios',
 'curva',
 'visto',
 'sextafeira',
 'hoje',
 'estimulado',
 'pelo',
 'bom',
 'humor',
 'ambiente',
 'externo',
 'recuo',
 'entanto',
 'não',
 'teve',
 'respaldo',
 'liquidez',
 'investidores',
 'preferindo',
 'aguardar',
 'eventos',
 'semana',
 'montar',
 'posições',
 'furou',
 'r',
 '550',
 'caindo',
 'mínima',
 'r',
 '54924',
 'mercado',
 'vista',
 'há',
 'pouco',
 'operador',
 'câmbio',
 'hideaki',
 'iha',
 'corretora',
 'fair',
 'atribui',
 'ajuste',
 'perda',
 'força',
 'há',
 'pouco',
 'índice',
 'dxy',
 'mas',
 'princ

In [302]:
#Juntando Título e descrição do target 1
target_1 = test.loc[train['Target']==1,:]
string_1 = ''
for texto in target_1['Descrição']:
    novo_texto=cleanup(str(texto).lower())
    string_1+=''.join(novo_texto)
    
for titulo in target_1['Titulo']:
    novo_titulo=cleanup(str(titulo).lower())
    string_1+=''.join(novo_titulo)
    
lista_1 =string_1.split()

#Retirando Stopwords

i = 0
while i<len(lista_1):
    if lista_1[i] == 'queda':
        del lista_1[i]
    #Se a palavra estiver na lista de stopwords, deleta
    if lista_1[i] in stopwords:
        del lista_1[i]
        
        if i != 0:
            i-=1
    else:
        i+=1
lista_1


['ouro',
 'fechou',
 'alta',
 'terçafeira',
 '9',
 'apoiado',
 'pela',
 'relativa',
 'fraqueza',
 'contra',
 'outras',
 'moedas',
 'fortes',
 'além',
 'disso',
 'mercado',
 'operou',
 'compasso',
 'espera',
 'pelos',
 'dados',
 'inflação',
 'consumidor',
 'nos',
 'estados',
 'unidos',
 'outubro',
 'vão',
 'ser',
 'divulgados',
 'quartafeira',
 '10',
 'comex',
 'divisão',
 'metais',
 'new',
 'york',
 'mercantile',
 'exchange',
 'mesmo',
 'retomada',
 'percepção',
 'risco',
 'exterior',
 'longo',
 'maior',
 'parte',
 'sessão',
 'ibovespa',
 'pouco',
 'pouco',
 'chegou',
 'quartafeira',
 '16',
 'sétimo',
 'avanço',
 'consecutivo',
 'r',
 '51279',
 'mínima',
 'dia',
 'fechamento',
 'parecendo',
 'mostrar',
 'fluxo',
 'permanece',
 'pé',
 'despeito',
 'orientação',
 'iniciou',
 'sessão',
 'dessa',
 'quartafeira',
 '6',
 'baixa',
 'moderada',
 'mas',
 'passou',
 'subir',
 'mercado',
 'interno',
 'abertura',
 'deve',
 'ter',
 'pesado',
 'igpdi',
 'acima',
 'mediana',
 'mercado',
 '210',
 'mar

In [303]:
#Juntando Título e Descrição do Target 2
target_2 = test.loc[train['Target']==2,:]
string_2 = ''
for texto in target_2['Descrição']:
    novo_texto=cleanup(str(texto).lower())
    string_2+=''.join(novo_texto)
    
for titulo in target_2['Titulo']:
    novo_titulo=cleanup(str(titulo).lower())
    string_2+=''.join(novo_titulo)


lista_2 = string_2.split()

#Retirando Stopwords
i = 0
while i<len(lista_2):
    if lista_2[i] == 'alta':
        del lista_2[i]
    #Se a palavra estiver na lista de stopwords, deleta
    if lista_2[i] in stopwords:
        del lista_2[i]
        
        if i != 0:
            i-=1
    else:
        i+=1
lista_2

    

['principais',
 'mercados',
 'acionários',
 'europa',
 'fecharam',
 'queda',
 'segundafeira',
 '29',
 'pesa',
 'perspectiva',
 'banco',
 'central',
 'europeu',
 'bce',
 'assim',
 'como',
 'federal',
 'reserve',
 'fed',
 'banco',
 'central',
 'norteamericano',
 'seguirão',
 'trajetória',
 'hawkish',
 'após',
 'comentários',
 'formuladores',
 'políticas',
 'dois',
 'bcs',
 'segunda',
 'reino',
 'unido',
 'feriado',
 'juros',
 'futuros',
 'operam',
 'leve',
 'baixa',
 'manhã',
 'desta',
 'terçafeira',
 '22',
 'acompanhando',
 'movimento',
 'ante',
 'real',
 'antes',
 'leilão',
 'ntnb',
 'lft',
 '11h',
 'além',
 'disso',
 'véspera',
 'ipca15',
 'mercado',
 'irá',
 'monitorar',
 'presidente',
 'banco',
 'central',
 'bc',
 'roberto',
 'campos',
 'neto',
 'participa',
 'evento',
 'são',
 'paulo',
 'são',
 'paulo',
 '30',
 'nov',
 'reuters',
 'deu',
 'salto',
 'mercado',
 'futuro',
 'chegou',
 'superar',
 '570',
 'reais',
 'fim',
 'manhã',
 'desta',
 'terçafeira',
 'acompanhando',
 'rali',
 'g

## Tabela de frequencias relativas e absolutas para cada target! 
Queremos a frequência de cada palavra

#### Cálculo da frequência para target = 0 ; NEUTRA

In [304]:
# serie_0 = pd.Series(string_0.split())
serie_0 = pd.Series(lista_0)
tabela_0 = serie_0.value_counts(True)
tabela_0

após           0.007399
mais           0.006461
r              0.006044
juros          0.005836
reuters        0.005523
                 ...   
retrucando     0.000104
derivativos    0.000104
dígito         0.000104
543            0.000104
261            0.000104
Length: 3526, dtype: float64

In [305]:
#Frequencia absoluta para Serie  0
tabela_abs_0 = serie_0.value_counts()
tabela_abs_0

após           71
mais           62
r              58
juros          56
reuters        53
               ..
retrucando      1
derivativos     1
dígito          1
543             1
261             1
Length: 3526, dtype: int64

#### Cálculo da frequência para target = 1; POSITIVA 

In [306]:
#serie_1 = pd.Series(string_1.split())
serie_1 = pd.Series(lista_1)
tabela_1 = serie_1.value_counts(True)
tabela_1

juros         0.008644
alta          0.007979
taxa          0.007979
após          0.006649
mercado       0.006649
                ...   
sofrem        0.000665
035           0.000665
conduzidas    0.000665
admitiram     0.000665
maionovas     0.000665
Length: 916, dtype: float64

In [258]:
#Frequencia absoluta para Serie  0
tabela_abs_1 = serie_1.value_counts()
tabela_abs_1

juros         13
alta          12
taxa          12
após          10
mercado       10
              ..
sofrem         1
035            1
conduzidas     1
admitiram      1
maionovas      1
Length: 916, dtype: int64

#### Cálculo da frequencia para target = 2 ; NEGATIVA

In [307]:
# serie_2 = pd.Series(string_2.split())
serie_2 = pd.Series(lista_2)
tabela_2 = serie_2.value_counts(True)
tabela_2

após          0.008673
banco         0.007143
reuters       0.007143
mais          0.006122
estados       0.006122
                ...   
figueiredo    0.000510
fator         0.000510
susan         0.000510
momento       0.000510
discutir      0.000510
Length: 1140, dtype: float64

In [308]:
#Frequencia absoluta para Serie  2
tabela_abs_2 = serie_2.value_counts()
tabela_abs_2

após          17
banco         14
reuters       14
mais          12
estados       12
              ..
figueiredo     1
fator          1
susan          1
momento        1
discutir       1
Length: 1140, dtype: int64

### Para fins de comparação, é necessário também unir todas as palavras das notícias em uma única string e montar as tabelas de frequências

In [309]:
#Transformando novamente as listas em strings:
textao_0 = ''
for palavra in lista_0: 
    textao_0 += palavra + ' '
#print(textao_0)
    

In [310]:
textao_1 = ''
for palavra in lista_1: 
    textao_1 += palavra + ' '
#print(textao_1)

In [311]:
textao_2 = ''
for palavra in lista_2: 
    textao_2 += palavra + ' '
#print(textao_2)

In [312]:
#Concatena tudiin agora
todas_ = textao_0+textao_1+textao_2

lista_todas = todas_.split()
serie_todas = pd.Series(lista_todas)

tabela_todas = serie_todas.value_counts(True)
tabela_abs_todas = serie_todas.value_counts()
tabela_abs_todas


após             98
mais             81
juros            77
reuters          72
mercado          71
                 ..
etanol            1
pesquisadores     1
expressão         1
supostos          1
discutir          1
Length: 4296, dtype: int64

## Montando o classificador; Calculando as Probabilidades

In [266]:
prob_neutra = len(serie_0)/len(serie_todas)
prob_positiva = len(serie_1)/len(serie_todas)
prob_negativa= len(serie_2)/len(serie_todas)

In [267]:
#P(palavra|target)
#noticia=cleanup(str(texto).lower())

test['Junção']=test['Titulo']+' '+test['Descrição']
i=0
for noticia in test['Junção']:
    nova_noticia = str(noticia).lower()
    nova_noticia = cleanup(nova_noticia)
    
    test['Junção'][i]=nova_noticia
    i+=1
    
test.head()

<ipython-input-267-deba66ed3ad6>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Junção'][i]=nova_noticia


,Categoria,Titulo,Descrição,Data,Pagina,Target,Unnamed: 6,Junção
0,financas,Ouro fecha em alta com câmbio favorável a comm...,"O ouro fechou em alta nesta terça-feira, 9, ap...",09/11/2021 15:46,185,2,NaN,ouro fecha em alta com câmbio favorável a comm...
1,financas,"Bolsas da Europa fecham em queda, com comentár...",Os principais mercados acionários da Europa fe...,29/08/2022 13:33,6,2,NaN,bolsas da europa fecham em queda com comentári...
2,financas,Juros ficam estáveis com mercado ponderando câ...,Os juros terminaram a sessão regular praticame...,23/03/2022 18:00,101,2,NaN,juros ficam estáveis com mercado ponderando câ...
3,financas,Juros: Alívio externo e queda do dólar abrem e...,"Os juros futuros terminaram a segunda-feira, 2...",25/07/2022 17:38,26,2,NaN,juros alívio externo e queda do dólar abrem es...
4,financas,"Ibovespa sobe 0,31%, a 115,1 mil pontos, melho...",Mesmo com a retomada da percepção de risco no ...,16/02/2022 18:44,124,2,NaN,ibovespa sobe 031 a 1151 mil pontos melhor nív...


In [268]:
#Aqui, separamos cada probabilidade em listas vazias para guardar as probabilidades condicionais de cada target.
probNeutra = []
probPos = []
probNeg = []

#Fazemos esse for pra calcular as probabilidades condicionais de acordo com cada junção = (Titulo + Descrição de cada linha)
#Além disso, usamos a aba de "Testes" pois queremos que o nosso classificador saiba como julgar cada notícia.
for noticia in test['Junção']:
    separacao = noticia.split()
    
    probJuncao_dado_Neutra = 1
    probJuncao_dado_Pos = 1
    probJuncao_dado_Neg = 1
    
    for palavra in separacao:
        
        #Se a palavra estiver na tabela com freqabs de notc neutras:
        #P(Palabra|FreqANeutra) = P(FreqNeutra|Palavra)*P(Palavra)/P(FreqAbsNeutra)
        if palavra in tabela_abs_0:
            probJuncao_dado_Neutra *= (tabela_abs_0[palavra]+1) / (tabela_abs_0.sum() + len(tabela_abs_todas))
        else:
            probJuncao_dado_Neutra *= (0+1)/(tabela_abs_0.sum() + len(tabela_abs_todas))
            
        #Se a palavra estiver na tabela c freq abs de notc positiva:
        if palavra in tabela_abs_1:
            probJuncao_dado_Pos *= (tabela_abs_1[palavra]+1) / (tabela_abs_1.sum() + len(tabela_abs_todas))
        else:
            probJuncao_dado_Pos*= (0+1)/(tabela_abs_1.sum() + len(tabela_abs_todas))
            
        #Se a palavra estiver na tabela c freqabs de notic negativa:
        if palavra in tabela_abs_2:
            probJuncao_dado_Neg *= (tabela_abs_2[palavra]+1) / (tabela_abs_2.sum() + len(tabela_abs_todas))
        else:
            probJuncao_dado_Neg *= (0+1)/(tabela_abs_2.sum() + len(tabela_abs_todas))
    probNeutra.append(probJuncao_dado_Neutra)
    probPos.append(probJuncao_dado_Pos)
    probNeg.append(probJuncao_dado_Neg)

        

Considerando o suavizador de Laplace, o denominador será descartado a fim de encontrar a target com maior probabilidade. Esse procedimento será feito para cada target.

In [293]:
#P(Neutra|Notícia):  
probNeutra_dado_noticia = []
for noticia_neutra in probNeutra:
    neutra = noticia_neutra*prob_neutra
    probNeutra_dado_noticia.append(neutra)

#P(Positiva|Notícia):
probPositiva_dado_noticia = []
for noticia_pos in probPos:
    positiva = noticia_pos*prob_positiva
    probPositiva_dado_noticia.append(positiva)
    
#P(Negativa|Notícia):
probNegativa_dado_noticia = []
for noticia_neg in probNeg:
    negativa = noticia_neg*prob_negativa
    probNegativa_dado_noticia.append(negativa)


Por fim, basta classificar cada notícia, com base na comparação da probabilidade de cada target

In [294]:
for i in range(len(probNeutra_dado_noticia)): #Todas as listas de probabilidade da célula anterior possuem o mesmo tamanho, 
                                              #utilizaremos a lista com a probabilidade da notícia ser neutra.
    
    if probNegativa_dado_noticia[i] > probPositiva_dado_noticia[i] and probNegativa_dado_noticia[i] > probNeutra_dado_noticia[i]:
        test.loc[i, 'Teste'] = 2
    
    elif probPositiva_dado_noticia[i] > probNegativa_dado_noticia[i] and probPositiva_dado_noticia[i] > probNeutra_dado_noticia[i]:
        test.loc[i, 'Teste'] = 1

    else: 
        test.loc[i, 'Teste'] = 0

test.head(50)

,Categoria,Titulo,Descrição,Data,Pagina,Target,Unnamed: 6,Junção,Teste
0,financas,Ouro fecha em alta com câmbio favorável a comm...,"O ouro fechou em alta nesta terça-feira, 9, ap...",09/11/2021 15:46,185,2,NaN,ouro fecha em alta com câmbio favorável a comm...,1.0
1,financas,"Bolsas da Europa fecham em queda, com comentár...",Os principais mercados acionários da Europa fe...,29/08/2022 13:33,6,2,NaN,bolsas da europa fecham em queda com comentári...,2.0
2,financas,Juros ficam estáveis com mercado ponderando câ...,Os juros terminaram a sessão regular praticame...,23/03/2022 18:00,101,2,NaN,juros ficam estáveis com mercado ponderando câ...,0.0
3,financas,Juros: Alívio externo e queda do dólar abrem e...,"Os juros futuros terminaram a segunda-feira, 2...",25/07/2022 17:38,26,2,NaN,juros alívio externo e queda do dólar abrem es...,2.0
4,financas,"Ibovespa sobe 0,31%, a 115,1 mil pontos, melho...",Mesmo com a retomada da percepção de risco no ...,16/02/2022 18:44,124,2,NaN,ibovespa sobe 031 a 1151 mil pontos melhor nív...,1.0
5,financas,"Juros: taxas operam em leve baixa, na esteira ...",Os juros futuros operam com leve baixa na manh...,22/02/2022 09:50,122,2,NaN,juros taxas operam em leve baixa na esteira do...,2.0
6,financas,"Dólar amaina com Precatórios e exterior, mas f...","O dólar furou os R$ 5,50, caindo à mínima de R...",09/11/2021 11:02,185,2,NaN,dólar amaina com precatórios e exterior mas fi...,1.0
7,financas,Dólar emenda 3º pregão seguido de queda e acum...,"O dólar à vista emendou nesta sexta-feira, 27,...",27/05/2022 17:55,62,2,NaN,dólar emenda 3º pregão seguido de queda e acum...,2.0
8,financas,Dólar recua contra real após relatório de empr...,Por Luana Maria Benedito SÃO PAULO (Reuters) –...,08/10/2021 10:44,203,2,NaN,dólar recua contra real após relatório de empr...,0.0
9,financas,Ibovespa inicia última semana do mês em alta d...,Mesmo com a relativa fraqueza vista neste come...,25/07/2022 17:37,26,2,NaN,ibovespa inicia última semana do mês em alta d...,0.0


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [295]:
comparacao = pd.crosstab(test.Teste, test.Target, normalize= True)  
comparacao

Target,0,1,2
Teste,,,
0.0,0.217105,0.039474,0.055921
1.0,0.263158,0.032895,0.029605
2.0,0.309211,0.036184,0.016447


In [296]:
neu_verdadeiros = comparacao.iloc[0,0]*100
print('A porcentagem de verdadeiros neutros é {:.2f} %'.format(neu_verdadeiros)) 

A porcentagem de verdadeiros neutros é 21.71 %


In [297]:
pos_verdadeiros = comparacao.iloc[1,1]*100
print('A porcentagem de verdadeiros positivos é {:.2f} %'.format(pos_verdadeiros)) 

A porcentagem de verdadeiros positivos é 3.29 %


In [298]:
neg_verdadeiros = comparacao.iloc[2,2]*100
print('A porcentagem de verdadeiros negativos é {:.2f} %'.format(neg_verdadeiros)) 

A porcentagem de verdadeiros negativos é 1.64 %


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**